In [1]:
!pwd

/workspaces/llm-zoomcanp


In [2]:
!pip install sentence-transformers==2.7.0



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


# 1) Q1
* Ans: 0.07

In [3]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_name = 'multi-qa-distilbert-cos-v1'

embedding_model = SentenceTransformer(model_name)

In [5]:
user_question = "I just discovered the course. Can I still join it?"

In [6]:
encoded = embedding_model.encode(user_question)

In [35]:
len(encoded)

768

In [7]:
# Q1 answer
encoded[0]

0.078222655

# Q2
* Ans: (375, 768)

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [10]:
documents_filtered = [d for d in documents if d["course"]=="machine-learning-zoomcamp"]

In [11]:
len(documents_filtered)

375

In [12]:
import numpy as np

In [13]:
embeddings = []

In [14]:
documents_filtered[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [15]:
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [16]:
from tqdm import tqdm

for d in tqdm(documents_filtered):
    qa_text = f'{d["question"]} {d["text"]}'
    encoded_qa_text = embedding_model.encode(qa_text)
    embeddings.append(encoded_qa_text)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:38<00:00,  9.67it/s]


In [17]:
import numpy as np
X = np.array(embeddings)

In [18]:
X.shape

(375, 768)

In [19]:
X

array([[ 0.08805905,  0.01559355,  0.07925582, ...,  0.00745118,
         0.00241914,  0.01147128],
       [ 0.07066919, -0.01930568,  0.07738373, ...,  0.04495561,
         0.01078552, -0.02316595],
       [ 0.10229158, -0.01663243,  0.03414485, ...,  0.03215314,
        -0.04317448, -0.04574372],
       ...,
       [-0.00768192,  0.0107567 ,  0.01190489, ...,  0.06066168,
        -0.03393206,  0.01605331],
       [ 0.13408281, -0.03755205,  0.01976525, ...,  0.02163627,
        -0.01921137,  0.03690192],
       [ 0.0438602 , -0.0130077 ,  0.06373122, ...,  0.03338452,
         0.0225854 , -0.06305878]], dtype=float32)

# Q3
* Ans: 0.65

In [20]:
scores = X.dot(encoded)
scores

array([ 2.89217263e-01,  4.35050726e-01,  2.20572561e-01,  1.28085926e-01,
        3.28754455e-01,  4.58999664e-01,  4.48930591e-01,  2.70170599e-01,
        2.13975310e-01,  3.39745760e-01,  1.63339168e-01,  3.15918088e-01,
        1.53621323e-02,  4.14240420e-01,  6.50657296e-01,  4.12786484e-01,
        2.02390462e-01,  4.71707582e-01,  3.74821007e-01,  3.53319108e-01,
        4.50558245e-01,  1.85430944e-01,  3.45526993e-01,  8.22948217e-02,
       -2.19077598e-02, -4.64250743e-02,  8.60690773e-02,  9.87197682e-02,
       -2.14188285e-02,  1.28751006e-02,  2.54870541e-02,  3.66866291e-02,
        4.56338078e-02, -2.05279887e-02,  1.82662532e-02,  3.39789167e-02,
        1.66510604e-03,  1.03209512e-02,  3.40170749e-02, -2.65231431e-02,
       -4.11909968e-02, -4.97983992e-02, -6.20346665e-02, -7.63561390e-03,
        4.61616889e-02,  6.95944112e-03,  4.77613509e-02, -9.97766852e-04,
       -5.60126789e-02, -5.96206151e-02,  2.05209196e-01, -1.16968602e-01,
       -2.34160442e-02, -

In [21]:
scores.shape

(375,)

In [22]:
scores.max()

0.6506573

# Q4
 * Ans: 0.93

In [23]:
# define function for search
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [24]:
# use for 1 qq using  simularity
search_engine = VectorSearchEngine(documents=documents_filtered, embeddings=X)
search_engine.search(encoded, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [25]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [26]:
ground_truth[0:1]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

In [27]:
g = ground_truth[0]
g

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [28]:
hits = 0
for elem in tqdm(ground_truth):
    encoded_q =  embedding_model.encode(elem['question'])
    true_answer = elem['document']
    search_results= search_engine.search(encoded_q, num_results=5)
    if true_answer in {r['id'] for r in search_results}:
        hits+=1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [02:22<00:00, 12.87it/s]


In [29]:
hits

1720

In [30]:
len(ground_truth)

1830

In [31]:
hit_rate = hits/len(ground_truth)
hit_rate

0.9398907103825137

# Q5

Start Elastic search service from the command line:
```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [34]:
# tbd

from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '1716cbe54043', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'lLYIqF4xTE6TnP7ETCGA5Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
 {'text': "The process is automated now, so you should receive the email eventually. If you haven’t, check your promotions tab in Gmail as well as spam.\nIf you unsubscribed from our newsletter, you won't get course related updates too.\nBut don't worry, it’s not a problem. To make sure you don’t miss anything, join the #course-ml-zoomcamp channel in Slack and our telegram channel with announcements. This is enough to follow the course.",
  'section': 'General course-related questions',
  'question': "I filled the form, but haven't received a confirmation email. Is it normal?",
  'course': 'machine-learning-zoomcamp',
  'id': '6ba259b1'},

In [48]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "id":  {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [49]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [50]:
#created the dense vector using the pre-trained model
operations = []
for doc in tqdm(documents_filtered):
    # Transforming the title into an embedding using the model
    qa_text = f'{doc["question"]} {doc["text"]}'
    encoded_qa_text = embedding_model.encode(qa_text)
    doc["text_vector"] = encoded_qa_text.tolist()
    operations.append(doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:36<00:00, 10.28it/s]


In [51]:
# add encodings to the index
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:08<00:00, 45.08it/s]


In [52]:
# from Q1
search_term = "I just discovered the course. Can I still join it?"
vector_search_term = embedding_model.encode(search_term)

In [71]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [72]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course","id"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'AKLStZABCodpLxrpbUmE',
  '_score': 0.82532895,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'}},
 {'_index': 'course-questions',
  '_id': 'A6LStZABCodpLxrpbUnI',
  '_score': 0.73585373,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Welcome to the course! Go to th

In [73]:
# Q5 ans: (real) id with the top score (0.82532895) is ee58a693
r = res["hits"]["hits"]
r[0]

{'_index': 'course-questions',
 '_id': 'AKLStZABCodpLxrpbUmE',
 '_score': 0.82532895,
 '_source': {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'}}

In [74]:
r[0].keys()

dict_keys(['_index', '_id', '_score', '_source'])

In [75]:
r[0]['_source'].keys()

dict_keys(['question', 'course', 'section', 'text', 'id'])

In [76]:
[elem['_source']['id'] for elem in r]

['ee58a693', '0a278fb2', '6ba259b1', '9f261648', 'e7ba6b8a']

# Q6

In [77]:
index_name

'course-questions'

In [78]:
# Q6 - checking hits from the ground truth at scale

hits_es = 0
for elem in tqdm(ground_truth):
    encoded_q =  embedding_model.encode(elem['question'])
    query = {
        "field": "text_vector",
        "query_vector": encoded_q,
        "k": 5,
        "num_candidates": 10000, 
    }   

    true_answer = elem['document']
    
    search_results =  es_client.search(index =index_name, knn=query, source=["text", "section", "question", "course","id"])
    
    if true_answer in {r['_source']['id'] for r in search_results["hits"]["hits"]}:
        hits_es+=1

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:09<00:00, 26.23it/s]


In [79]:
hits_es

1720

In [81]:
hit_ratio_es = hits_es / len(ground_truth)
hit_ratio_es

0.9398907103825137